#  Kmeans 示例

使用TensorFlow实现Kmeans算法，实验数据是yann的mnist手写数字数据库，用来做手写数字分类。

In [16]:
from __future__ import print_function

import numpy as np
import tensorflow as tf
from tensorflow.contrib.factorization import KMeans

import os

os.environ["CUDA_VISIBLE_DEVICES"]=""

In [17]:
# 利用TensorFlow中的例程中的读取数据的模块，读取数据
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data", one_hot=True)
full_data_x = mnist.train.images

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [18]:
# 设置参数
num_steps = 50
batch_size = 1024
k = 25
num_classes= 10
num_features= 28 * 28

X = tf.placeholder(tf.float32, shape=[None, num_features])
Y = tf.placeholder(tf.float32, shape=[None, num_classes])

kmeans = KMeans(inputs=X, num_clusters= k, distance_metric='cosine',use_mini_batch=True)
# 使用在TensorFlow中定义的KMeans类，初始化一个KMeans对象，这个类的init函数的参数由如下几个部分构成
# __init__(
#     inputs,  聚类数据
#     num_clusters,  聚类个数
#     initial_clusters=RANDOM_INIT,
#     distance_metric=SQUARED_EUCLIDEAN_DISTANCE,
#     use_mini_batch=False,
#     mini_batch_steps_per_iteration=1,
#     random_seed=0,
#     kmeans_plus_plus_num_retries=2,
#     kmc2_chain_length=200
# )

In [29]:
# 创建计算图
# 使用KMeans类中的training_graph创建一个聚类图，该函数的返回值是一个元组
# A tuple consisting of: 
# all_scores: 一个维度是 (num_input, num_clusters)的矩阵，其中的值是一个每个数据与每个聚类中心的距离
# cluster_idx：一个向量，每个元素值是每个数据对应的聚类中心的下标
# scores：与cluster_idx类似，每个元素是每个数据对应聚类中心的距离
# cluster_centers_initialized：标量，表示这个聚类中心是否被初始化
# init_op：一个聚类的操作
# training_op：运行一个迭代的训练操作
(all_scores, cluster_idx, scores, cluster_centers_initialized,init_op,train_op)= kmeans.training_graph()
# with tf.Session() as temp:
#     print(temp.run(cluster_idx))
cluster_idx = cluster_idx[0]
avg_distance = tf.reduce_mean(scores)
init_vars = tf.global_variables_initializer()

In [32]:
# 开启一个Session
sess = tf.Session()
# 运行初始化
sess.run(init_vars, feed_dict= {X: full_data_x})
sess.run(init_op, feed_dict={X: full_data_x})

# 训练
for i in range(1, num_steps+1):
    _, d, idx = sess.run([train_op, avg_distance, cluster_idx],feed_dict={X: full_data_x})
    if i % 10 == 0 or i == 1:
        print("Step %i, Avg Distance :%f" % (i, d))

Step 1, Avg Distance :0.341471
Step 10, Avg Distance :0.221609
Step 20, Avg Distance :0.220328
Step 30, Avg Distance :0.219776
Step 40, Avg Distance :0.219419
Step 50, Avg Distance :0.219154


In [38]:
# 给每一个centeroid一个标签，计算每个标签的数量，对每个数据找距离当前数据最近的中心，这个最近中心的label就是这个数据的label
counts = np.zeros(shape=(k, num_classes))
for i in range(len(idx)):
    counts[idx[i]] += mnist.train.labels[i]
    
labels_map = [np.argmax(c) for c in counts]
labels_map = tf.convert_to_tensor(labels_map)
## 我认为这里已经结束了，为啥还要进行测试